This is file runs the main calculation for the flexible length TPS simulation. It requires the file `alanine_dipeptide_tps_equil.nc`, which is written in the notebook `alanine_dipeptide_tps_first_traj.ipynb`.

In this file, you will learn:
* how to set up and run a flexible length TPS simulation

NB: This is a long calculation. In practice, it would be best to export the Python from this notebook, remove the `live_visualization`, and run non-interactively on a computing node.

In [1]:
import openpathsampling as paths

## Load engine, trajectory, and states from file

In [2]:
old_storage = paths.Storage("tps_nc_files/alanine_dipeptide_tps_equil.nc", "r")

In [3]:
engine = old_storage.engines[0]
alpha = old_storage.volumes.find('alpha')
beta = old_storage.volumes.find('beta')
traj = old_storage.samplesets[len(old_storage.samplesets)-1][0].trajectory
phi = old_storage.cvs.find('phi')
psi = old_storage.cvs.find('psi')
template = old_storage.snapshots[0]

In [4]:
print engine.name

300K


## TPS

As always, the process for setting up a simulation is:

1. Create a `network`
2. Create a `move_scheme`
3. Set up `initial_conditions`
4. Create the `PathSampling` object and run it.

Since we already created all the input to these when we set up the first trajectory, we can load use the versions we loaded above.

In [5]:
network = paths.TPSNetwork(alpha, beta)

In [6]:
scheme = paths.OneWayShootingMoveScheme(network, selector=paths.UniformSelector(), engine=engine)

In [7]:
initial_conditions = scheme.initial_conditions_from_trajectories(traj)

# finished generating: still missing 0 samples
1


In [8]:
sampler = paths.PathSampling(storage=paths.Storage("tps_nc_files/alanine_dipeptide_tps.nc", "w", template),
                             move_scheme=scheme,
                             sample_set=initial_conditions)
sampler.live_visualization = paths.LiveVisualization(network, phi, psi, [-3.14, 3.14], [-3.14, 3.14])

Note: 10000 steps will take a long time. If you just want to run a little bit, reduce this number.

In [9]:
sampler.run(200)

DONE! Completed 200 Monte Carlo cycles.


With this done, you can go on to do the flexible-length parts of the analysis in `alanine_dipeptide_tps_analysis.ipynb`.

In [11]:
from collections import Counter

In [29]:
q = Counter()

In [28]:
sampler.storage.movechanges.set_caching(True)

In [30]:
for ch in sampler.storage.movechanges:
    q.update([ch.__class__])

In [33]:
q

Counter({openpathsampling.movechange.EmptyMoveChange: 1,
         openpathsampling.movechange.AcceptedSampleMoveChange: 119,
         openpathsampling.movechange.RejectedSampleMoveChange: 81,
         openpathsampling.movechange.RandomChoiceMoveChange: 600,
         openpathsampling.movechange.PathSimulatorMoveChange: 200})